# 002 JOIN

```SQL
-- Limpieza de tablas previas para evitar conflictos
DROP TABLE IF EXISTS stock;
DROP TABLE IF EXISTS ventas_dia;
DROP TABLE IF EXISTS productos;

-- 1. Tabla de Catálogo de Productos
CREATE TABLE productos (
    producto_id INTEGER PRIMARY KEY,
    nombre_producto VARCHAR(50),
    categoria VARCHAR(50)
);

-- 2. Tabla de Ventas Diarias
CREATE TABLE ventas_dia (
    venta_id INTEGER PRIMARY KEY,
    producto_id INTEGER, -- No ponemos Foreign Key para poder insertar el ID 99 (error)
    cantidad INTEGER
);

-- 3. Tabla de Stock en Bodega
CREATE TABLE stock (
    producto_id INTEGER,
    cantidad_disponible INTEGER
);

-- Insertar en Catálogo
INSERT INTO productos (producto_id, nombre_producto, categoria)
VALUES
    (10, 'Espresso', 'Bebida'),
    (20, 'Latte', 'Bebida'),
    (30, 'Muffin', 'Comida'),
    (40, 'Galleta', 'Comida');

-- Insertar en Ventas del Día
-- Nota el ID 99 que no existe en el catálogo
INSERT INTO ventas_dia (venta_id, producto_id, cantidad)
VALUES
    (501, 10, 5),
    (502, 20, 3),
    (503, 99, 1);

-- Insertar en Stock
-- Nota el ID 50 que es un producto nuevo que no está en el catálogo
INSERT INTO stock (producto_id, cantidad_disponible)
VALUES
    (10, 100),
    (20, 50),
    (30, 20),
    (50, 10);

SELECT * FROM productos;
SELECT * FROM ventas_dia;
SELECT * FROM stock;
```

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_productos = pd.read_csv('Data/002/productos.csv')
df_stock = pd.read_csv('Data/002/stock.csv')
df_ventas = pd.read_csv('Data/002/ventas_dia.csv')

df_productos.head()

,producto_id,nombre_producto,categoria
0,10,Espresso,Bebida
1,20,Latte,Bebida
2,30,Muffin,Comida
3,40,Galleta,Comida


In [3]:
df_stock.head()

,producto_id,cantidad_disponible
0,10,100
1,20,50
2,30,20
3,50,10


In [4]:
df_ventas.head()

,venta_id,producto_id,cantidad
0,501,10,5
1,502,20,3
2,503,99,1
